In [160]:
import numpy as np
import pandas as pd

## Load and explore data

In [161]:
training_data = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
# using pre-split dataset: 60,000 training examples and 10,000 test examples
training_data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [162]:
# convert to np array
data = np.array(training_data)
# m = number of training examples, n = number of features (pixels in this case) + 1
m, n = data.shape

## Transform data to correct format for ANN

In [191]:
# transpose so each row is an example (except first row which = label)
data = data.T
# create Y_train (labels) and X_train
Y_train = data[0]
X_train = data[1:].astype(np.float32)

In [192]:
# normalise X_train 
X_train /= 255

In [164]:
print(f"m = {m}, n = {n - 1}")
print(f"X_train shape = {X_train.shape}")
print(f"Y_train shape = {Y_train.shape}")

m = 60000, n = 784
X_train shape = (784, 60000)
Y_train shape = (60000,)


In [182]:
# one hot encode Y_train for calculation of cost function later

def one_hot(Y):
    one_hot_Y = np.zeros((10, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y


In [166]:
# check one_hot_Y is working by comparing first 5 values of Y_train with first 5 columns of one_hot_Y

Y_train[:5]

array([5, 0, 4, 1, 9])

In [167]:
one_hot(Y_train)[:,:5]

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

## Initialise parametres (weights and biases) 

In [186]:
first_layer_nodes = 10
second_layer_nodes = 10 
'''
my initial plan was to not make this a hardcoded 2 x 10 layer network and have it more dynamic. 
I will play with this in due course but would like to just get it running first!
'''

# this is the init_params function I wrote before discovering that there is a much more sensible version 
def init_params_old():
    W_1 = np.random.rand(10, n-1) - 0.5 
    # weights for input layer to first layer
    b_1 = np.random.rand(10,1) - 0.5
    # bias for first layer of 10 nodes (if first_layer_nodes = 10)
    W_2 = np.random.rand(first_layer_nodes, second_layer_nodes) - 0.5
    # weights between first and second layer (10 -> 10)
    b_2 = np.random.rand(second_layer_nodes,1) - 0.5
    # biases for second layer 
    
    # np.random.rand() returns random num between 0 and 1 so -0.5 distributes between -0.5 and 0.5
    
    return W_1, b_1, W_2, b_2


def init_params():
    W1 = np.random.randn(first_layer_nodes, 784) * np.sqrt(2/784)
    b1 = np.zeros((first_layer_nodes, 1))
    W2 = np.random.randn(10, 10) * np.sqrt(2/10)
    b2 = np.zeros((10, 1))
    return W1, b1, W2, b2


## Forward prop function + activation functions

In [181]:
# activation functions

def ReLU(N): 
    return np.maximum(0, N) 
    # np.maximum compares two arrays and returns the highest value in each position
    # so if number in < 0, 0 will be returned (i.e. ReLU)

def softmax(Z):
    Z = Z - np.max(Z, axis=0, keepdims=True)
    expZ = np.exp(Z)
    return expZ / np.sum(expZ, axis=0, keepdims=True)

In [170]:
# test activation functions work as expected

test_1 = np.random.rand(10,1) - 0.5
print(f"ReLU: {ReLU(test_1)}")
print(f"Softmax: {softmax(test_1)}\n Sum of Softmax (should = 1) = {np.sum(softmax(test_1))}")

ReLU: [[0.49166336]
 [0.30986108]
 [0.        ]
 [0.33909465]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.20391278]
 [0.21169147]
 [0.0965825 ]]
Softmax: [[0.14530541]
 [0.12115074]
 [0.08494718]
 [0.12474468]
 [0.0589795 ]
 [0.07692902]
 [0.07126787]
 [0.10897159]
 [0.10982256]
 [0.09788144]]
 Sum of Softmax (should = 1) = 1.0000000000000002


In [171]:
# forward prop

def forward_prop(X_train, W_1, b_1, W_2, b_2):
    Z_1 = np.dot(W_1, X_train) + b_1
    A_1 = ReLU(Z_1)
    Z_2 = np.dot(W_2, A_1) + b_2
    A_2 = softmax(Z_2)
    return Z_1, A_1, Z_2, A_2

## Back prop

In [172]:
# need derivative functions for activation functions 

def deriv_ReLU(N):
    return np.where(N < 0, 0, 1)

In [173]:
def back_prop(Z_1, A_1, Z_2, A_2, W_2, X, Y): 
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ_2 = A_2 - one_hot_Y
    dW_2 = 1 / m * np.dot(dZ_2, A_1.T)
    db_2 = 1 / m * np.sum(dZ_2, axis=1, keepdims=True)
    dZ_1 = np.dot(W_2.T, dZ_2) * deriv_ReLU(Z_1)
    dW_1 = 1 / m * np.dot(dZ_1, X.T)
    db_1 = 1 / m * np.sum(dZ_1, axis=1, keepdims=True)
    return dW_1, db_1, dW_2, db_2

In [174]:
def update_params(W_1, b_1, W_2, b_2, dW_1, db_1, dW_2, db_2, alpha):
    W_1 -= alpha * dW_1
    b_1 -= alpha * db_1
    W_2 -= alpha * dW_2
    b_2 -= alpha * db_2
    return W_1, b_1, W_2, b_2

In [175]:
# this is actually completely redundant - I started confusing the cost function with accuracy 

def get_cost(A_2, Y):
    return np.sum((A_2 - Y) **2) / Y.size

A = np.array([0,1,2,3,4,5])
B = np.array([0,1,2,5,7,5])
print(get_cost(A, B))

2.1666666666666665


In [176]:
def get_predictions(A_2):
    # this should convert A_2 probabilities into a one hot prediction
    return np.argmax(A_2, axis=0)

In [177]:
def get_accuracy(predictions, Y):
    return np.mean(predictions == Y)

In [178]:
C = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1, 0.1])
D = np.array([8])
acc = get_accuracy(get_predictions(C), D)
print(acc)

1.0


## Gradient descent

In [184]:
def gradient_descent(X, Y, iterations, alpha):
    W_1, b_1, W_2, b_2 = init_params()
    for i in range(iterations):
        Z_1, A_1, Z_2, A_2 = forward_prop(X, W_1, b_1, W_2, b_2)
        dW_1, db_1, dW_2, db_2 = back_prop(Z_1, A_1, Z_2, A_2, W_2, X, Y)
        W_1, b_1, W_2, b_2 = update_params(W_1, b_1, W_2, b_2, dW_1, db_1, dW_2, db_2, alpha)
        
        if i % 50 == 0: 
            print(f"Iteration: {i}")
            print(f"Accuracy: {get_accuracy(get_predictions(A_2), Y)}")
            preds = get_predictions(A_2)
            print("Most common predicted class:", np.bincount(preds).argmax())
            print("Prediction counts (0-9):", np.bincount(preds, minlength=10))


    return W_1, b_1, W_2, b_2

In [196]:
W_1, b_1, W_2, b_2 = gradient_descent(X_train, Y_train, 400, 0.1)

Iteration: 0
Accuracy: 0.0898
Most common predicted class: 1
Prediction counts (0-9): [ 1238 25343  2340   628     0 15644  5701  6597  2322   187]
Iteration: 50
Accuracy: 0.7191
Most common predicted class: 7
Prediction counts (0-9): [6391 7839 5567 7047 2025 4216 6137 7871 5472 7435]
Iteration: 100
Accuracy: 0.817
Most common predicted class: 1
Prediction counts (0-9): [6178 7191 5683 6252 5505 4655 6270 5865 5638 6763]
Iteration: 150
Accuracy: 0.8573166666666666
Most common predicted class: 1
Prediction counts (0-9): [6200 7113 5675 6115 5997 4816 6197 5976 5657 6254]
Iteration: 200
Accuracy: 0.8754
Most common predicted class: 1
Prediction counts (0-9): [6187 7059 5719 6093 6058 4910 6126 6055 5697 6096]
Iteration: 250
Accuracy: 0.8849333333333333
Most common predicted class: 1
Prediction counts (0-9): [6174 7015 5738 6078 6062 4982 6093 6082 5711 6065]
Iteration: 300
Accuracy: 0.8911833333333333
Most common predicted class: 1
Prediction counts (0-9): [6154 6976 5742 6078 6051 5052

# Test data

In [199]:
test_data = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')
test_data = np.array(test_data)

# transpose so each row is an example (except first row which = label)
test_data = test_data.T
# create Y_train (labels) and X_train
Y_test = data[0]
X_test = data[1:].astype(np.float32)

X_test /= 255

In [207]:
# predict - essesntially forward_prop but only needs to return A_2
def predict(X, W_1, b_1, W_2, b_2):
    Z_1 = np.dot(W_1, X) + b_1
    A_1 = ReLU(Z_1)
    Z_2 = np.dot(W_2, A_1) + b_2
    A_2 = softmax(Z_2)
    return A_2

test_predictions_softmax = predict(X_test, W_1, b_1, W_2, b_2)

In [211]:
test_predictions = get_predictions(test_predictions_softmax)
test_accuracy = get_accuracy(test_predictions, Y_test)
print(f"Accuracy on test dataset = {round(test_accuracy*100, 2)}%.")

Accuracy on test dataset = 89.91%.


## Testing my own handwritten number

In [228]:
# my handwritten number 4

from PIL import Image
import pandas as pd

def preprocess_digit(path):
    # load image and convert to grayscale
    img = Image.open(path).convert('L')

    # resize to 28x28
    img = img.resize((28, 28), Image.LANCZOS)

    # convert to numpy array
    img_array = np.array(img)

    # invert colors (MNIST: white digit on black)
    img_array = 255 - img_array

    # normalise to 0–1
    img_array = img_array / 255.0

    # Flatten to (784, 1)
    img_array = img_array.reshape(784, 1)

    return img_array

X_test_my_number = preprocess_digit('/kaggle/input/number4/44491078-dba3-45f0-827b-1cacdff3ab3e1.jpg')

In [231]:
idx = np.argmax(predict(X_test_my_number, W_1, b_1, W_2, b_2))
print(idx)

4


### Problem log: 
- Multiple typos due to underscores
- Dimensions of several arrays off - need to keep track
- Finally code ran but for some reason by iteration 50 only 2 is being predicted

- DIDN'T NORMALISE X_TRAIN!!! This took me a while. 

### Lessons learned: 
- Naming variables - avoid underscores if not needed! I cannot believe how many errors I had to change because of "d_W1" to "dW_1".
- Write down complicated maths on paper notes to avoid getting lost in the calculus while trying to write code
- Diagrams are very helpful
- Keep track of dimensions (m x n) for dot products / matrix multiplication!
- May have been more efficient to write out basic function inputs and outputs are start to conceptualise structure


### Overall, 
Fun little challenge to jog my memory of basic backpropogation and really get into the weeds of it.